📌 Cập Nhật Dataset & Huấn Luyện TimeSformer

In [ ]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from torch.utils.data import Dataset, DataLoader

model_name = "facebook/timesformer-base-finetuned-k400"
model = TimesformerForVideoClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
processor = AutoImageProcessor.from_pretrained(model_name)

labels = {"đấm": 0, "đá": 1, "tát": 2} 

def extract_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        print(f"⚠ Video {video_path} quá ngắn!")
        return None

    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Chuyển về RGB
        frames.append(frame)

    cap.release()
    return frames if len(frames) == num_frames else None

class ActionVideoDataset(Dataset):
    def __init__(self, data_folder, labels):
        self.data_folder = data_folder
        self.labels = labels
        self.video_paths = []
        self.targets = []

        for action, label in labels.items():
            action_folder = os.path.join(data_folder, action)
            if os.path.exists(action_folder):
                for video in os.listdir(action_folder):
                    if video.endswith(".mp4"):
                        video_path = os.path.join(action_folder, video)
                        self.video_paths.append(video_path)
                        self.targets.append(label)

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.targets[idx]

        frames = extract_frames(video_path, num_frames=8)
        if frames is None:
            return None

        inputs = processor(images=frames, return_tensors="pt")
        return inputs["pixel_values"].squeeze(0), torch.tensor(label)


train_dataset = ActionVideoDataset("data", labels)
train_dataset = [d for d in train_dataset if d is not None] 
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=inputs)
        loss = loss_fn(outputs.logits, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"🔄 Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")

# Lưu mô hình
model.save_pretrained("custom_timesformer")
processor.save_pretrained("custom_timesformer")
print("✅ Huấn luyện xong! Đã lưu mô hình tại custom_timesformer")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🔄 Epoch 1/5 - Loss: 143.0471
🔄 Epoch 2/5 - Loss: 29.8161
🔄 Epoch 3/5 - Loss: 14.9409
🔄 Epoch 4/5 - Loss: 7.8184
🔄 Epoch 5/5 - Loss: 4.7769
✅ Huấn luyện xong! Đã lưu mô hình tại custom_timesformer


📌 Cập Nhật Code Dự Đoán

In [ ]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms
from transformers import TimesformerForVideoClassification, AutoImageProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"
model = TimesformerForVideoClassification.from_pretrained("custom_timesformer").to(device)
processor = AutoImageProcessor.from_pretrained("custom_timesformer")


labels = ["đấm", "đá", "tát"]

def extract_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        print(f"⚠ Video {video_path} quá ngắn!")
        return None

    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Chuyển về RGB
        frames.append(frame)

    cap.release()
    return frames if len(frames) == num_frames else None

def predict_action(video_path):
    print(f"📂 Đang xử lý video: {video_path}")
    frames = extract_frames(video_path, num_frames=8)
    
    if frames is None:
        print("⚠ Video quá ngắn hoặc lỗi khi trích xuất frames.")
        return

    inputs = processor(images=frames, return_tensors="pt")
    inputs["pixel_values"] = inputs["pixel_values"].to(device)  # Đưa dữ liệu lên GPU

    print("✅ Đã tiền xử lý xong, bắt đầu dự đoán...")
    
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    print("🔢 Giá trị logits:", logits)
    print("🎯 Chỉ số dự đoán:", predicted_class)

    if 0 <= predicted_class < len(labels):
        print("📌 Dự đoán hành động:", labels[predicted_class])
    else:
        print("⚠ Lỗi: Chỉ số dự đoán ngoài phạm vi!", predicted_class)

predict_action("V_797.mp4")


📂 Đang xử lý video: V_797.mp4
✅ Đã tiền xử lý xong, bắt đầu dự đoán...
🔢 Giá trị logits: tensor([[13.5167, 10.6946,  6.0627,  2.0314, -0.1815,  1.3605, -1.0458, -1.6472,
          0.5926, -0.2519, -2.9618, -2.2580,  2.1829,  1.4996, -0.1884, -1.4981,
          3.3528, -1.1448, -0.2744, -1.7093, -3.5220,  0.9008, -2.2963,  2.6589,
         -0.5216,  2.2699,  0.8419, -1.0576, -2.9878, -1.3719, -2.8797, -2.8307,
         -0.7263, -0.2985, -0.8822,  0.0927, -0.4938,  1.1167, -0.4299, -1.0720,
         -0.2360,  5.0088, -1.1312,  0.3184,  2.8360, -4.9479, -2.3153, -1.7634,
         -0.6309, -0.3694,  0.1598,  2.6668, -0.7356,  4.5414, -0.8498, -1.2402,
          0.5067,  1.3860, -3.7504, -0.7423, -2.1778,  1.9309, -1.7978,  0.3151,
         -2.2947,  4.4684,  3.9735,  2.1701,  1.7811, -0.4944, -2.0105, -0.2254,
         -0.8658, -2.3823,  0.7104,  0.3265, -2.6117, -4.4250,  3.7640,  0.7856,
          0.3645,  0.2973, -1.6184, -2.8684, -3.8761,  3.4028,  3.8132,  3.2711,
         -2.9112, -2